# Data – English to Spanish Dataset

In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
from datasets import load_dataset
from transformers import AutoTokenizer

/Users/spencerkarofsky/miniconda3/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset = load_dataset('Helsinki-NLP/opus_books', 'en-es')

In [3]:
tokenizer = AutoTokenizer.from_pretrained('Helsinki-NLP/opus-mt-en-es')

/Users/spencerkarofsky/miniconda3/lib/python3.12/site-packages/transformers/models/marian/tokenization_marian.py:177: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [4]:
class TranslationDataset(Dataset):
    def __init__(self, split, max_length=128):
        self.data = dataset[split]
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sample = self.data[idx]
        source = sample['translation']['en']
        target = sample['translation']['es']

        source_enc = tokenizer(source, padding='max_length', truncation=True, max_length=self.max_length, return_tensors='pt')
        target_enc = tokenizer(target, padding='max_length', truncation=True, max_length=self.max_length, return_tensors='pt')

        return {
            'input_ids': source_enc['input_ids'].squeeze(0),
            'attention_mask': source_enc['attention_mask'].squeeze(0),
            'labels': target_enc['input_ids'].squeeze(0)
        }

In [7]:
train_dataset = TranslationDataset('train')
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

batch = next(iter(train_loader))
print('X shape:', batch['input_ids'].shape)
print('y shape:', batch['labels'].shape)

X shape: torch.Size([32, 128])
y shape: torch.Size([32, 128])
